In [1]:
import pandas as pd
from hnmchallenge.data_reader import DataReader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np



from hnmchallenge.constant import *
import datetime

In [2]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

In [3]:
dataset = StratifiedDataset()
dr = DataReader()

### times MB items

In [4]:
# creating H&M features in holdin 
feature = dataset.get_holdin()

In [5]:
item_feature=feature.drop(["sales_channel_id","price"], axis=1)

In [6]:
item_feature

,t_dat,customer_id,article_id
0,2018-09-20,0,0
1,2018-09-20,0,0
2,2019-08-05,0,0
3,2019-08-16,0,644
4,2019-10-06,0,1952
...,...,...,...
10955395,2020-09-22,1136204,17478
10955396,2020-09-22,1136205,10033
10955397,2020-09-22,1136205,7580
10955398,2020-09-22,1136205,1765


In [7]:
duplicated_rows = item_feature[item_feature.duplicated(subset=[DEFAULT_USER_COL, DEFAULT_ITEM_COL])]
count_mb = duplicated_rows.groupby(DEFAULT_ITEM_COL).count()
count_mb = count_mb.reset_index()[[DEFAULT_ITEM_COL, "t_dat"]].rename(
    columns={"t_dat": "count"}
)

In [8]:
count_mb

,article_id,count
0,0,545
1,1,295
2,2,153
3,3,405
4,4,2110
...,...,...
17995,21951,1
17996,21953,1
17997,21956,2
17998,21997,2


In [9]:
item_feature=pd.merge(count_mb,item_feature, on="article_id",how="outer")

In [10]:
item_feature = item_feature.drop(["t_dat", "customer_id"], axis=1)

In [11]:
item_feature = item_feature.drop_duplicates().fillna(0)

In [12]:
item_feature

,article_id,count
0,0,545.0
3301,1,295.0
5387,2,153.0
6470,3,405.0
8346,4,2110.0
...,...,...
10955395,21552,0.0
10955396,22065,0.0
10955397,22066,0.0
10955398,20576,0.0


### Popularity > 1 sep

In [13]:
base_df = feature.drop(["sales_channel_id","price"], axis=1)

In [14]:
item_feature1=base_df[base_df["t_dat"]>="2020-09-01"]

In [15]:
item_feature1

,t_dat,customer_id,article_id
457,2020-09-14,19,4705
512,2020-09-01,23,2873
513,2020-09-01,23,16951
514,2020-09-01,23,6749
515,2020-09-01,23,14886
...,...,...,...
10955395,2020-09-22,1136204,17478
10955396,2020-09-22,1136205,10033
10955397,2020-09-22,1136205,7580
10955398,2020-09-22,1136205,1765


In [16]:
popularity = item_feature1.groupby(DEFAULT_ITEM_COL).count()["customer_id"].rename("popularity").to_frame().reset_index()

In [17]:
popularity

,article_id,popularity
0,1,1
1,4,9
2,5,41
3,6,8
4,7,278
...,...,...
15589,22062,1
15590,22063,1
15591,22064,1
15592,22065,1


In [18]:
item_features=item_feature.set_index("article_id").join(
    popularity.set_index('article_id'), on='article_id')

In [19]:
item_features = item_features.fillna(0)

In [20]:
item_features =  item_features.reset_index()

In [21]:
item_features

,article_id,count,popularity
0,0,545.0,0.0
1,1,295.0,1.0
2,2,153.0,0.0
3,3,405.0,0.0
4,4,2110.0,9.0
...,...,...,...
21126,21552,0.0,1.0
21127,22065,0.0,1.0
21128,22066,0.0,1.0
21129,20576,0.0,1.0


In [24]:
dr = DataReader()
item_features.to_feather(
        dr.get_preprocessed_data_path() / "stratified_holdin_item_dataset.feather"
    )

In [25]:
dataset.get_features_
holdin_item()

,article_id,count,popularity
0,0,545.0,0.0
1,1,295.0,1.0
2,2,153.0,0.0
3,3,405.0,0.0
4,4,2110.0,9.0
...,...,...,...
21126,21552,0.0,1.0
21127,22065,0.0,1.0
21128,22066,0.0,1.0
21129,20576,0.0,1.0
